# GG P2 Project

* istXXXXXXX1 FirstName1 LastName1 (30%)
  
* istXXXXXXX2 FirstName2 LastName2 (33%)
  
* istXXXXXXX3 FirstName3 LastName3 (37%)

Prof. FirstName LastName (and Prof. FirstName LastName)

Lab Shift number: PBXX

## PART I – Original Database Schema (SQLite)

##### E-R Diagram

![E-R Diagram](soccer.drawio.png "E-R Diagram")

#### 1. The Original Database Schema

Study the E-R Diagram above corresponding to the original database schema.


Lets download the SQLite database backup/dump!

In [ ]:
!wget -c https://github.com/bdist/is-labs-data/raw/refs/heads/main/soccer/database.sqlite?download= -O /home/jovyan/data/database.sqlite

#### 2. SQL Queries

Consider the E-R Diagram above and write SQL queries for each of the following information needs.

In [ ]:
%reload_ext sql
%config SqlMagic.displaycon = 0
%config SqlMagic.displaylimit = 100
%config SqlMagic.feedback = 0
%sql sqlite:////home/jovyan/data/database.sqlite --alias sqlite

1. What is the maximum number of goals recorded in a match? Output a single number.

In [ ]:
%%sql sqlite


2. List each league name and its number of teams sorted from high to low.

In [ ]:
%%sql sqlite


3. Who are the GoalKeepers (GK) that have played all the away games for their team? Output the `team_long_name` and `player_name` (assuming `match.away_player_1` is the goalkeeper column).

In [ ]:
%%sql sqlite


## PART II - New Database Schema (PostgreSQL)

#### 0. Migrate the SQLite Database to PostgreSQL

Instead of writing tedious migration code ourselves, we will be using the `pgloader` CLI tool to get started quickly. The migration task is setup using a `command-file`. Lets see inside that file.

In [ ]:
!cat ./command-file.txt

The `pgloader` tool will take care of everything automatically including the foreign key constraints (which SQLite does not enforce by default). Lets run it.

In [ ]:
!pgloader ./command-file.txt

Lets take a closer look into the `match` table.

In [ ]:
%load_ext sql
%config SqlMagic.displaycon = 0
%config SqlMagic.displaylimit = 100
%config SqlMagic.feedback = 0
%sql postgresql+psycopg://db:db@postgres/db --alias psql

In [ ]:
%sqlcmd columns -t match

Notice how `pgloader` even assigned the safest most-compatible data types for each column so that no data is lost in the migration.

##### 0.1. Introducing a limitation of the original database schema

The `match` table still does not look good to us... Can you try to write a SQL query for the following information need?

1. List all the teams that 'Cristiano Ronaldo' has played for (regardless of starting position)

In [ ]:
%%sql psql
-- WILL NOT BE EVALUATED - We will be implementing a solution for this problem in the next section.

#### 1. Functions, Stored Procedures, and Triggers

Provide the SQL instructions corresponding to each of the following tasks:

1. Create a new `match_player` table to store the data for the starting players (11) for each match (currently stored in the columns `match.home_player_DD` and `match.away_player_DD`). We are going to copy the data contained in the table `match` to `match_player` but we want to use a new column for the player position `position`.

In [ ]:
%%sql psql
DROP TABLE IF EXISTS match_player;

CREATE TABLE
  match_player ();

2. Populate the new table `match_player` using the data contained in the table `match` using a function or stored procedure.

In [ ]:
%%sql psql
DELETE FROM match_player;

CREATE OR REPLACE PROCEDURE populate_match_player()
LANGUAGE plpgsql
AS $$
BEGIN

END;
$$;

CALL populate_match_player();

#### 2. SQL Queries

Consider the new `match_player` table above and write SQL queries for each of the following information needs.

1. List all the teams that 'Cristiano Ronaldo' has played for (regardless of starting position)

In [ ]:
%%sql psql


2. Who are the players that have played all the away games for their team? Output the `team_long_name` and `player_name`.

In [ ]:
%%sql psql


3. What is the match with the highest number of goals? Output `team_long_name` for both home and away teams, and the total number of goals for each team.

In [ ]:
%%sql psql


#### 3. Query Optimization

Consider the new `player_attributes` table that contains the largest amount of rows.

1. Create the most appropriate index(es) for the following query: 

In [ ]:
%%sql psql
CREATE INDEX

In [ ]:
%%sql psql
SELECT
  MAX(overall_rating)
FROM
  player_attributes
WHERE
  attacking_work_rate = 'high';

2. Create the most appropriate index(es) for the following query: 

In [ ]:
%%sql psql
CREATE INDEX

In [ ]:
%%sql psql
SELECT
  player_api_id,
  preferred_foot,
  MIN(crossing),
  MIN(potential)
FROM
  player_attributes
GROUP BY
  player_api_id,
  preferred_foot
HAVING
  MIN(crossing) = 80
  AND MIN(potential) >= 90

## PART III - The Web App (Flask + PostgreSQL)

#### 1. SQL View

1. Create a SQL View that lists the player names sorted by the most recent `player_attributes.date` first.

In [ ]:
%%sql psql
CREATE OR REPLACE VIEW
  players_index_view

2. Create the most appropriate index(es) for the SQL view you just created. (Note: PostgreSQL does not have support for materialized views)

In [ ]:
%%sql psql
CREATE INDEX

3. Please test your view using the following query.

In [ ]:
%%sql psql
SELECT
  *
FROM
  players_index_view
LIMIT
  20;

#### 2. Create the players_index page on the Web App

Requirements:
- Should use the SQL View declared above
- Do not list more than 20 players.

1. Copy your players_index function from app.py to the cell below

```python
@app.route("/", methods=("GET",))
@app.route("/accounts", methods=("GET",))
@limiter.limit("1 per second")
def account_index():
    """Show all the accounts, most recent first."""

    with pool.connection() as conn:
        with conn.cursor() as cur:
            accounts = cur.execute(
                """
                SELECT account_number, branch_name, balance
                FROM account
                ORDER BY account_number DESC;
                """,
                {},
            ).fetchall()
            log.debug(f"Found {cur.rowcount} rows.")

    return render_template("account/index.html", accounts=accounts)
```

2. Copy your final HTML template for this view onto the cell below

```html
{% extends "base.html" %}
{% block header %}
    <h1>
        {% block title %}
            Accounts
        {% endblock title %}
    </h1>
{% endblock header %}
{% block content %}
    {% for account in accounts %}
        <article class="post">
            <header>
                <div>
                    <h1>{{ account['account_number'] }}</h1>
                    <div class="about">in {{ account['branch_name'] }}</div>
                </div>
                <a class="action"
                   href="{{ url_for('account_update_view', account_number=account['account_number']) }}">Edit</a>
            </header>
            <p class="body">€ {{ account['balance'] }}</p>
        </article>
        {% if not loop.last %}<hr />{% endif %}
    {% endfor %}
{% endblock content %}
```

#### 3. Create the player_update_view and player_update_save routes on the Web App

Requirements:

- The app.route for player_update_view should take a player_api_id and show a nice editable HTML form.
             
- The app.route for player_update_save should be called on form submission and insert the new data into player_attributes. 

- On success, should redirect to the players_index

- On failure, should raise an appropriate exception with a error message

1. Copy your player_update_view function from app.py to the cell below

2. Copy your final HTML template for this view onto the cell below

3. Copy your player_update_save function from app.py to the cell below